In [1]:
import os
os.chdir('/nfs/homedirs/ayle/guided-research/SNIP-it/')

In [2]:
import torch
from torchvision import datasets, transforms
import foolbox as fb
from experiments.main import load_checkpoint
from models import GeneralModel
from models.statistics.Metrics import Metrics
from utils.config_utils import *
from utils.model_utils import *
from utils.system_utils import *
from utils.attacks_utils import get_attack
from torch.utils.data.dataset import Dataset

In [3]:
arguments = dict({
'eval_freq': 1000,  # evaluate every n batches
    'save_freq': 1e6,  # save model every n epochs, besides before and after training
    'batch_size': 512,  # size of batches, for Imagenette 128
    'seed': 465,  # random seed
    'max_training_minutes': 6120 , # one hour and a 45 minutes max, process killed after n minutes (after finish of epoch)
    'plot_weights_freq': 50, # plot pictures to tensorboard every n epochs
    'prune_freq': 4, # if pruning during training: how long to wait before starting
    'prune_delay': 8, # "if pruning during training: 't' from algorithm box, interval between pruning events, default=0
    'prune_to': 1,
    'epochs': 80,
    'rewind_to': 0, # rewind to this epoch if rewinding is done
    'snip_steps': 5, # 's' in algorithm box, number of pruning steps for 'rule of thumb', TODO
    'pruning_rate': 0.0 , # pruning rate passed to criterion at pruning event. however, most override this
    'growing_rate': 0.0000 , # grow back so much every epoch (for future criterions)
    'pruning_limit': 0.98,  # Prune until here, if structured in nodes, if unstructured in weights. most criterions use this instead of the pruning_rate
    'learning_rate': 2e-3,
    'grad_clip': 10,
    'grad_noise': 0 , # added gaussian noise to gradients
    'l2_reg': 5e-5 , # weight decay
    'l1_reg': 0 , # l1-norm regularisation
    'lp_reg': 0 , # lp regularisation with p < 1
    'l0_reg': 1.0 , # l0 reg lambda hyperparam
    'hoyer_reg': 0.001 , # hoyer reg lambda hyperparam
    'beta_ema': 0.999 , # l0 reg beta ema hyperparam

    'loss': 'CrossEntropy',
    'optimizer': 'ADAM',
    'model': 'ResNet18',  # ResNet not supported with structured
    'data_set': 'CIFAR10',
    'ood_data_set': 'SVHN',
    'prune_criterion': 'SNIPitDuring',  # options: SNIP, SNIPit, SNIPitDuring, UnstructuredRandom, GRASP, HoyerSquare, IMP, // SNAPit, StructuredRandom, GateDecorators, EfficientConvNets, GroupHoyerSquare
    'train_scheme': 'DefaultTrainer' , # default: DefaultTrainer
    'attack': 'FGSM',
    'epsilon': 4,
    'eval_ood_data_sets': ['CIFAR100', 'SVHN', 'GAUSSIAN', 'OODOMAIN'],
    'eval_attacks': ['FGSM', 'L2FGSM'],
    'eval_epsilons': [4, 6, 12, 24, 48],

    'device': 'cuda',
    'results_dir': "tests",

    'checkpoint_name': None,
    'checkpoint_model': None,

    'disable_cuda_benchmark': 1 , # speedup (disable) vs reproducibility (leave it)
    'eval': 0,
    'disable_autoconfig': 0 , # for the brave
    'preload_all_data': 0 , # load all data into ram memory for speedups
    'tuning': 0 , # splits trainset into train and validationset, omits test set

    'track_weights': 0 , # "keep statistics on the weights through training
    'disable_masking': 1 , # disable the ability to prune unstructured
    'enable_rewinding': 0 , # enable the ability to rewind to previous weights
    'outer_layer_pruning': 1, # allow to prune outer layers (unstructured) or not (structured)
    'first_layer_dense': 0,
    'random_shuffle_labels': 0  ,# run with random-label experiment from zhang et al
    'l0': 0,  # run with l0 criterion, might overwrite some other arguments
    'hoyer_square': 0, # "run in unstructured DeephoyerSquare criterion, might overwrite some other arguments
    'group_hoyer_square': 0 ,# run in unstructured Group-DeephoyerSquare criterion, might overwrite some other arguments

    'disable_histograms': 0,
    'disable_saliency': 0,
    'disable_confusion': 0,
    'disable_weightplot': 0,
    'disable_netplot': 0,
    'skip_first_plot': 0,
    'disable_activations': 0,
    
#     'input_dim': [1, 28, 28],
#       'output_dim': 10,
#       'hidden_dim': [512],
#       'N': 60000,
    
    'input_dim': [3, 32, 32],
      'output_dim': 10,
      'hidden_dim': [512],
      'N': 60000
})

In [4]:
DATASET_PATH = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/data'

In [5]:
metrics = Metrics()
out = metrics.log_line
metrics._batch_size = arguments['batch_size']
metrics._eval_freq = arguments['eval_freq']
set_results_dir(arguments["results_dir"])

In [6]:
model: GeneralModel = find_right_model(
        NETWORKS_DIR, arguments['model'],
        device=arguments['device'],
        hidden_dim=arguments['hidden_dim'],
        input_dim=arguments['input_dim'],
        output_dim=arguments['output_dim'],
        is_maskable=arguments['disable_masking'],
        is_tracking_weights=arguments['track_weights'],
        is_rewindable=arguments['enable_rewinding'],
        is_growable=arguments['growing_rate'] > 0,
        outer_layer_pruning=arguments['outer_layer_pruning'],
        maintain_outer_mask_anyway=(
                                       not arguments['outer_layer_pruning']) and (
                                           "Structured" in arguments['prune_criterion']),
        l0=arguments['l0'],
        l0_reg=arguments['l0_reg'],
        N=arguments['N'],
        beta_ema=arguments['beta_ema'],
        l2_reg=arguments['l2_reg']
).to(arguments['device'])

Ignored arguments in ResNet18: {'hidden_dim': [512]}


In [7]:
load_checkpoint(arguments, model, out)

In [8]:
device = arguments['device']

In [9]:
# get criterion
criterion = find_right_model(
    CRITERION_DIR, arguments['prune_criterion'],
    model=model,
    limit=arguments['pruning_limit'],
    start=0.5,
    steps=arguments['snip_steps'],
    device=arguments['device'],
    arguments=arguments
)

# load data
train_loader, test_loader = find_right_model(
    DATASETS, arguments['data_set'],
    arguments=arguments
)

# load OOD data
_, ood_loader = find_right_model(
    DATASETS, arguments['ood_data_set'],
    arguments=arguments
)

# get loss function
loss = find_right_model(
    LOSS_DIR, arguments['loss'],
    device=device,
    l1_reg=arguments['l1_reg'],
    lp_reg=arguments['lp_reg'],
    l0_reg=arguments['l0_reg'],
    hoyer_reg=arguments['hoyer_reg']
)

# get optimizer
optimizer = find_right_model(
    OPTIMS, arguments['optimizer'],
    params=model.parameters(),
    lr=arguments['learning_rate'],
    weight_decay=arguments['l2_reg'] if not arguments['l0'] else 0
)

run_name = f'_model={arguments["model"]}_dataset={arguments["data_set"]}_ood-dataset={arguments["ood_data_set"]}' + \
           f'_attack={arguments["attack"]}_epsilon={arguments["epsilon"]}_prune-criterion={arguments["prune_criterion"]}' + \
           f'_pruning-limit={arguments["pruning_limit"]}_prune-freq={arguments["prune_freq"]}_prune-delay={arguments["prune_delay"]}' + \
           f'_rewind-to={arguments["rewind_to"]}_train-scheme={arguments["train_scheme"]}_seed={arguments["seed"]}'


# build trainer
trainer = find_right_model(
    TRAINERS_DIR, arguments['train_scheme'],
    model=model,
    loss=loss,
    optimizer=optimizer,
    device=device,
    arguments=arguments,
    train_loader=train_loader,
    test_loader=test_loader,
    ood_loader=ood_loader,
    metrics=metrics,
    criterion=criterion,
    run_name=run_name
)

trainer.train()

Ignored arguments in SNIPitDuring: {'start': 0.5, 'arguments': {'eval_freq': 1000, 'save_freq': 1000000.0, 'batch_size': 512, 'seed': 465, 'max_training_minutes': 6120, 'plot_weights_freq': 50, 'prune_freq': 4, 'prune_delay': 8, 'prune_to': 1, 'epochs': 80, 'rewind_to': 0, 'snip_steps': 5, 'pruning_rate': 0.0, 'growing_rate': 0.0, 'pruning_limit': 0.98, 'learning_rate': 0.002, 'grad_clip': 10, 'grad_noise': 0, 'l2_reg': 5e-05, 'l1_reg': 0, 'lp_reg': 0, 'l0_reg': 1.0, 'hoyer_reg': 0.001, 'beta_ema': 0.999, 'loss': 'CrossEntropy', 'optimizer': 'ADAM', 'model': 'ResNet18', 'data_set': 'CIFAR10', 'ood_data_set': 'SVHN', 'prune_criterion': 'SNIPitDuring', 'train_scheme': 'DefaultTrainer', 'attack': 'FGSM', 'epsilon': 4, 'eval_ood_data_sets': ['CIFAR100', 'SVHN', 'GAUSSIAN', 'OODOMAIN'], 'eval_attacks': ['FGSM', 'L2FGSM'], 'eval_epsilons': [4, 6, 12, 24, 48], 'device': 'cuda', 'results_dir': 'tests', 'checkpoint_name': None, 'checkpoint_model': None, 'disable_cuda_benchmark': 1, 'eval': 0, '

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.7792969  |  0.6394091   |    0.7396053    |  1.0133932  | 0.6595244  |   0.7087159    |        0.5519818        |   0.8806378   |    0.6176817    |   0.4933204    |    3499948.0     |    0.0000000   
$ |  sparse/node  |  sparse/hm  |  sparse/log_disk_size  |  time/gpu_time  |  time/flops_per_sample  |  time/flops_log_cum  |  cuda/ram_footprint  |  time/batch_time  |  
$ |   0.0000000   |  0.0000000  |       19.8013823       |   119.6562574   |       38800136.0        |      12.8924958      |     237171712.0      |     0.0305371     |
Training... 97/98




EPOCH 9  




Training... 0/98

Evaluating... 19/20

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.8613281  |  0.4223829   |    0.4245156    |  0.8052488  | 0.7421530  |   0.5072666    |        0.5277401        |   0.7898264   |    0.6966359    |   0.589

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.9257812  |  0.2204132   |    0.3088574    |  0.8396135  | 0.7453010  |   0.4096879    |        0.5214097        |   0.6534926   |    0.6998884    |   0.5888501    |    2506956.0     |    0.7396803   
$ |  sparse/node  |  sparse/hm  |  sparse/log_disk_size  |  time/gpu_time  |  time/flops_per_sample  |  time/flops_log_cum  |  cuda/ram_footprint  |  time/batch_time  |  
$ |   0.0000000   |  0.7424800  |       18.4556043       |   120.0688507   |       38800136.0        |      13.4036134      |     239531008.0      |     0.0413752     |
Training... 97/98



EPOCH 14  




Training... 0/98

Evaluating... 19/20

> PERFORMING ADV ATTACK FGSM
>

Training... 97/98



EPOCH 18  




Training... 0/98

Evaluating... 19/20

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.9472656  |  0.1763916   |    0.2063713    |  0.8861176  | 0.7524414  |   0.3405713    |        0.5150793        |   0.4339815   |    0.

...

Training... 97/98



EPOCH 60  




Training... 0/98

Evaluating... 19/20

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.9746094  |  0.0905665   |    0.0680890    |  1.0319860  | 0.7832778  |   0.1909292    |        0.5184858        |   0.4055471   |    0.

Training... 97/98



EPOCH 66  




Training... 0/98

Evaluating... 19/20

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.9765625  |  0.0730425   |    0.0979509    |  1.1082882  | 0.7752872  |   0.1874145    |        0.5188132        |   0.3551559   |    0.

Training... 97/98



EPOCH 72  




Training... 0/98

Evaluating... 19/20

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.9843750  |  0.0723044   |    0.0650167    |  1.1284435  | 0.7748851  |   0.1755228    |        0.5080480        |   0.3685953   |    0.

Training... 97/98



EPOCH 78  




Training... 0/98

Evaluating... 19/20

> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
> PERFORMING ADV ATTACK FGSM
$  acc/train  |  loss/train  |  entropy/train  |  loss/test  |  acc/test  |  entropy/test  |  adv_success_rate/test  |  entropy/ood  |  entropy/auroc  |  entropy/aupr  |  sparse/l0_norm  |  sparse/weight 
$  0.9804688  |  0.0680621   |    0.0711143    |  1.0374100  | 0.7844152  |   0.1844242    |        0.4990005        |   0.4967711   |    0.

<Figure size 432x288 with 0 Axes>

In [10]:
results = {'train_acc': trainer.train_acc, 'test_acc': trainer.test_acc, 'sparsity': trainer.sparsity,
               'filename': DATA_MANAGER.stamp}

In [11]:
for attack in arguments['eval_attacks']:
    for epsilon in arguments['eval_epsilons']:
        out("Attack {}".format(attack))
        # build tester
        tester = find_right_model(
            TESTERS_DIR, 'AdversarialEvaluation',
            attack=attack,
            model=trainer._model,
            device=device,
            arguments=None,
            test_loader=test_loader,
        )

        out("Epsilon {}".format(str(epsilon)))
        res = tester.evaluate(epsilon=epsilon)

        for key, value in res.items():
            results[key] = value

Attack FGSM
Epsilon 4
> PERFORMING ADV ATTACK FGSM
EPSILON 4 Successes attack 0.51171875 same prediction as model 0.693359375 bounds adver -2.1179039478302 2.640000104904175 norm 0.8684213546803221
> PERFORMING ADV ATTACK FGSM
EPSILON 4 Successes attack 0.48828125 same prediction as model 0.708984375 bounds adver -2.1179039478302 2.640000104904175 norm 0.8686539231566712
> PERFORMING ADV ATTACK FGSM
EPSILON 4 Successes attack 0.4921875 same prediction as model 0.71875 bounds adver -2.1179039478302 2.640000104904175 norm 0.8684298675507307
> PERFORMING ADV ATTACK FGSM
EPSILON 4 Successes attack 0.505859375 same prediction as model 0.705078125 bounds adver -2.1179039478302 2.640000104904175 norm 0.8684800404589623
> PERFORMING ADV ATTACK FGSM
EPSILON 4 Successes attack 0.51953125 same prediction as model 0.73046875 bounds adver -2.1179039478302 2.640000104904175 norm 0.8687249530339614
> PERFORMING ADV ATTACK FGSM
EPSILON 4 Successes attack 0.501953125 same prediction as model 0.703125 b

EPSILON 12 Successes attack 0.798828125 same prediction as model 0.421875 bounds adver -2.1179039478302 2.640000104904175 norm 2.6040391251444817
> PERFORMING ADV ATTACK FGSM
EPSILON 12 Successes attack 0.798828125 same prediction as model 0.408203125 bounds adver -2.1179039478302 2.640000104904175 norm 2.6032155980356038
> PERFORMING ADV ATTACK FGSM
EPSILON 12 Successes attack 0.8046875 same prediction as model 0.40234375 bounds adver -2.1179039478302 2.640000104904175 norm 2.6038755723275244
> PERFORMING ADV ATTACK FGSM
EPSILON 12 Successes attack 0.82421875 same prediction as model 0.369140625 bounds adver -2.1179039478302 2.640000104904175 norm 2.603074150159955
> PERFORMING ADV ATTACK FGSM
EPSILON 12 Successes attack 0.826171875 same prediction as model 0.419921875 bounds adver -2.1179039478302 2.640000104904175 norm 2.603672088123858
> PERFORMING ADV ATTACK FGSM
EPSILON 12 Successes attack 0.84375 same prediction as model 0.361328125 bounds adver -2.1179039478302 2.64000010490417

EPSILON 48 Successes attack 0.947265625 same prediction as model 0.240234375 bounds adver -2.1179039478302 2.640000104904175 norm 10.391146261245012
> PERFORMING ADV ATTACK FGSM
EPSILON 48 Successes attack 0.974609375 same prediction as model 0.267578125 bounds adver -2.1179039478302 2.640000104904175 norm 10.385987436398864
> PERFORMING ADV ATTACK FGSM
EPSILON 48 Successes attack 0.962890625 same prediction as model 0.234375 bounds adver -2.1179039478302 2.640000104904175 norm 10.390425585210323
> PERFORMING ADV ATTACK FGSM
EPSILON 48 Successes attack 0.951171875 same prediction as model 0.259765625 bounds adver -2.1179039478302 2.640000104904175 norm 10.387021725997329
> PERFORMING ADV ATTACK FGSM
EPSILON 48 Successes attack 0.9742646813392639 same prediction as model 0.20955882966518402 bounds adver -2.1179039478302 2.640000104904175 norm 10.384902680621428
Attack L2FGSM
Epsilon 4
> PERFORMING ADV ATTACK L2FGSM
EPSILON 4 Successes attack 0.21484375 same prediction as model 0.9921875

EPSILON 12 Successes attack 0.216796875 same prediction as model 0.982421875 bounds adver -2.1179039478302 2.640000104904175 norm 0.04697302617387322
> PERFORMING ADV ATTACK L2FGSM
EPSILON 12 Successes attack 0.240234375 same prediction as model 0.974609375 bounds adver -2.1179039478302 2.640000104904175 norm 0.04686257804246452
> PERFORMING ADV ATTACK L2FGSM
EPSILON 12 Successes attack 0.23828125 same prediction as model 0.98046875 bounds adver -2.1179039478302 2.640000104904175 norm 0.04693879215847119
> PERFORMING ADV ATTACK L2FGSM
EPSILON 12 Successes attack 0.275390625 same prediction as model 0.978515625 bounds adver -2.1179039478302 2.640000104904175 norm 0.046752397839156856
> PERFORMING ADV ATTACK L2FGSM
EPSILON 12 Successes attack 0.240234375 same prediction as model 0.96875 bounds adver -2.1179039478302 2.640000104904175 norm 0.0468984933113461
> PERFORMING ADV ATTACK L2FGSM
EPSILON 12 Successes attack 0.234375 same prediction as model 0.982421875 bounds adver -2.11790394783

EPSILON 48 Successes attack 0.322265625 same prediction as model 0.904296875 bounds adver -2.1179039478302 2.640000104904175 norm 0.1881530831742566
> PERFORMING ADV ATTACK L2FGSM
EPSILON 48 Successes attack 0.306640625 same prediction as model 0.90234375 bounds adver -2.1179039478302 2.640000104904175 norm 0.18812174489721656
> PERFORMING ADV ATTACK L2FGSM
EPSILON 48 Successes attack 0.2890625 same prediction as model 0.92578125 bounds adver -2.1179039478302 2.640000104904175 norm 0.18779660528889508
> PERFORMING ADV ATTACK L2FGSM
EPSILON 48 Successes attack 0.306640625 same prediction as model 0.892578125 bounds adver -2.1179039478302 2.640000104904175 norm 0.1881165661616251
> PERFORMING ADV ATTACK L2FGSM
EPSILON 48 Successes attack 0.349609375 same prediction as model 0.896484375 bounds adver -2.1179039478302 2.640000104904175 norm 0.18751744349719957
> PERFORMING ADV ATTACK L2FGSM
EPSILON 48 Successes attack 0.306640625 same prediction as model 0.900390625 bounds adver -2.11790394

In [12]:
with torch.no_grad():
    for ood_data_set in arguments['eval_ood_data_sets']:
        out("OOD Dataset: {}".format(ood_data_set))
        # load data
        _, test_loader = find_right_model(
            DATASETS, arguments['data_set'],
            arguments=arguments
        )

        # load OOD data
        _, ood_loader = find_right_model(
            DATASETS, ood_data_set,
            arguments=arguments
        )
        # build tester
        tester = find_right_model(
            TESTERS_DIR, 'OODEvaluation',
            model=trainer._model,
            device=device,
            arguments=None,
            test_loader=test_loader,
            ood_loader=ood_loader,
            ood_dataset=ood_data_set
        )
        res = tester.evaluate()

        for key, value in res.items():
            results[key] = value

OOD Dataset: CIFAR100
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
OOD Dataset: SVHN
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: gitignored/data/train_32x32.mat
Using downloaded and verified file: gitignored/data/test_32x32.mat
OOD Dataset: GAUSSIAN
Files already downloaded and verified
Files already downloaded and verified
OOD Dataset: OODOMAIN
Files already downloaded and verified
Files already downloaded and verified


In [13]:
class DS(Dataset):

    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        self.mean = [0.485, 0.456, 0.406]  # avg 0.449
        self.std = [0.229, 0.224, 0.225]  # avg 0.226
        self.transforms = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(mean=self.mean, std=self.std)
            ]
        )

    def __getitem__(self, item):
        image = self.images[item] / 255
        image = self.transforms(image.transpose((1, 2, 0)))
        return image.to(torch.float32), torch.tensor(self.labels[item], dtype=torch.float32)

    def __len__(self):
        return len(self.images)

with torch.no_grad():
    if arguments["data_set"] == "CIFAR10":
        ds_path = os.path.join(DATASET_PATH, "cifar10_corrupted")
        for ds_dataset_name in os.listdir(ds_path):
            npz_dataset = np.load(os.path.join(ds_path, ds_dataset_name))

            ds_dataset = DS(npz_dataset["images"], npz_dataset["labels"])
            ds_loader = torch.utils.data.DataLoader(
                ds_dataset,
                batch_size=arguments['batch_size'],
                shuffle=False,
                pin_memory=True,
                num_workers=4
            )

            # build tester
            tester = find_right_model(
                TESTERS_DIR, 'DSEvaluation',
                model=trainer._model,
                device=device,
                arguments=None,
                test_loader=test_loader,
                ds_loader=ds_loader,
                ds_dataset=ds_dataset_name.split('.')[0]
            )
            res = tester.evaluate()

            for key, value in res.items():
                results[key] = value

            break

In [14]:
results

{'train_acc': 0.98046875,
 'test_acc': 0.7839384191176471,
 'sparsity': 0.979576565696821,
 'filename': '2021-05-21_21.36.52_model=ResNet18_dataset=CIFAR10_ood-dataset=SVHN_attack=FGSM_epsilon=4_prune-criterion=SNIPitDuring_pruning-limit=0.98_prune-freq=4_prune-delay=8_rewind-to=0_train-scheme=DefaultTrainer_seed=465',
 'success_rate_FGSM_4': 0.5070886954665184,
 'success_rate_FGSM_6': 0.62373046875,
 'success_rate_FGSM_12': 0.8076573997735977,
 'success_rate_FGSM_24': 0.9213177859783173,
 'success_rate_FGSM_48': 0.9611155778169632,
 'success_rate_L2FGSM_4': 0.23019301444292067,
 'success_rate_L2FGSM_6': 0.23379480689764023,
 'success_rate_L2FGSM_12': 0.24441636055707933,
 'success_rate_L2FGSM_24': 0.265355009585619,
 'success_rate_L2FGSM_48': 0.3111270681023598,
 'conf_auroc': 0.8463280590805133,
 'conf_aupr': 0.9517338160642659,
 'brier_score': 0.31159532,
 'entropy': 0.1744104,
 'auroc_CIFAR100': 0.8736717100000001,
 'aupr_CIFAR100': 0.9023482379400285,
 'entropy_CIFAR100': 0.858138